# RAG Pipeline for Indian Legal Documents

## 1. Installation

In [1]:
!pip install langchain langchain-community pypdf chromadb jupyter langchain-google-genai langchain-chroma sentence-transformers nbconvert -q

## 2. Configuration

In [2]:
import os
from getpass import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")

## 3. Data Ingestion

In [3]:
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

DATA_PATH = "data"
CHROMA_PATH = "chroma"

def ingest_data():
    if not os.path.exists(DATA_PATH):
        os.makedirs(DATA_PATH)
        print(f"Created directory: {DATA_PATH} Awaiting PDF files...")
        return

    loader = PyPDFDirectoryLoader(DATA_PATH)
    documents = loader.load()
    
    if not documents:
        print("No documents found in the 'data' directory.")
        return

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    
    print(f"Splitting {len(documents)} documents into {len(chunks)} chunks.")

    embeddings = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")
    
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
    
    print(f"Successfully ingested data and created a vector store at: {CHROMA_PATH}" )

if __name__ == "__main__":
    ingest_data()


Splitting 658 documents into 2332 chunks.


C:\Users\ankit\AppData\Local\Temp\ipykernel_33828\1509366030.py:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")


modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.7k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

NotFoundError: Error getting collection: Collection [409ca36d-3763-4a45-a43c-3322cc54b6b3] does not exists.

## 5. RAG Chain with Gemini

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def create_rag_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
    embeddings = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")
    vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)

    retriever = vectorstore.as_retriever()

    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer the question. "
        "If you don't know the answer, just say that you don't know. "
        "Use three sentences maximum and keep the answer concise."
        "\n\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{input}")])

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    return rag_chain

rag_chain = create_rag_chain()

query = "What is the main point of the document?"
response = rag_chain.invoke({"input": query})

print(response["answer"])
